In [1]:
!pip install -U bitsandbytes
!pip install accelerate
!pip install PyPDF2
!pip install pinecone-client
!pip install fpdf
!pip install huggingface_hub
!pip install transformers
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=9a19d0e53ebc40f906c47bda005becf08b4f88c8fd4f37018096bdf879c0a13e
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [2]:
import os
from PyPDF2 import PdfReader
import math
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)
from getpass import getpass
import torch
from torch.cuda.amp import autocast

In [3]:
# Function to read the PDF content
def read_pdf(file_path):
    """
    Reads text from a PDF file.

    Args:
        file_path (str): Path to the PDF file.

    Returns:
        str: Combined text of all pages.
    """
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text.strip()

In [4]:
# Function to split content into chunks
def create_chunks(text, chunk_size, overlap):
    """
    Splits text into chunks with overlapping context.

    Args:
        text (str): The input text.
        chunk_size (int): The maximum size of each chunk (characters).
        overlap (int): The number of overlapping characters between chunks.

    Returns:
        list: List of text chunks.
    """
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        word_length = len(word) + 1
        if current_length + word_length > chunk_size:
            chunks.append(" ".join(current_chunk))
            overlap_words = (
                current_chunk[-math.ceil(overlap / len(current_chunk)) :]
                if current_chunk
                else []
            )
            current_chunk = overlap_words
            current_length = sum(len(w) + 1 for w in current_chunk)

        current_chunk.append(word)
        current_length += word_length

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [5]:
file_path = "/content/AFFAIRE C.P. ET M.N. c. FRANCE.pdf"
document_text = read_pdf(file_path)
chunks = create_chunks(document_text, chunk_size=3000, overlap=500)

In [6]:
len(chunks)

17

In [7]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("Enter your Hugging Face API token: ")

Enter your Hugging Face API token: ··········


In [8]:
model_name = "facebook/mbart-large-50-many-to-one-mmt"
# model = MBartForConditionalGeneration.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
model = MBartForConditionalGeneration.from_pretrained(model_name)
# tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [10]:
# Function to translate chunks into English
def translate_chunks(chunks, batch_size):
    """
    Translates a list of text chunks into English using MBart50.

    Args:
        chunks (list): List of text chunks in the source language.
        batch_size (int): Number of chunks to process in a single batch.

    Returns:
        list: Translated text chunks in English.
    """
    translations = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(model.device)

        with autocast():
            generated_tokens = model.generate(**encoded_input, max_length=128, num_beams=1)

        translations.extend(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))
    return translations

In [11]:
translated_chunks = translate_chunks(chunks, batch_size=8)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
<ipython-input-10-dc36a21ebc50>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


In [12]:
for i, translation in enumerate(translated_chunks, 1):
    print(f"Chunk {i} Translation:\n{translation}\n")

Chunk 1 Translation:
PART FIVE Case C.P. and M.N. v. FRANCE (Applications Nos. 56513/17 and 56515/17) JUDGMENT Art 8 Privacy and family life Refusal by the domestic courts to examine the action of the applicant, claiming to be the biological father of a child, challenging the legally established paternity with a view to establishing his own, in application of the rules for calculating the five-year limitation period combined with the obligation to bring the child before the courts in the case Applicant having failed

Chunk 2 Translation:
The applicant’s former partner had recognised the latter before his birth on 4 December 2007. JUDGMENT C.P. and M.N. v. FRANCE 25. At the beginning of March 2012, the applicant left her former partner and entered into a civil solidarity pact (PACS) with the applicant on 14 March 2012. 6. On 12 December 2012, the applicant applied to the Family Court (JAF) for the measures relating to the two children to be fixed and requested the establishment of an al

In [13]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

In [14]:
pc = Pinecone(api_key=getpass("Enter your Pinecone API token: "))

Enter your Pinecone API token: ··········


In [15]:
index_name = "legal-llm"
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

index = pc.Index(index_name)

In [16]:
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

embeddings = embedding_model.encode(translated_chunks)

for i, chunk in enumerate(translated_chunks):
    index.upsert(vectors=[(f"doc_{i}", embeddings[i], {"text": chunk})])

print("Translated chunks embedded and stored in Pinecone.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Translated chunks embedded and stored in Pinecone.


In [17]:
from huggingface_hub import login

login(token=getpass("Enter your Hugging Face access token: "))

Enter your Hugging Face access token: ··········


In [18]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

# Configure the model for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# Load the model with the BitsAndBytesConfig
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    quantization_config=bnb_config,
    device_map="auto",
)

# Create a pipeline for text generation
llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0


In [19]:
# Prompt template
PROMPT_TEMPLATE = """You are a highly skilled legal expert with deep knowledge of law and legal cases.
You provide accurate, well-reasoned, and reliable answers based strictly on the given legal documents.

Context:
{context}

Question:
{question}

Instructions:
- Answer the question strictly based on the given context.
- If the context does not contain enough information, state that explicitly.
- Provide detailed legal reasoning where necessary.
- Keep the response clear and professional.
"""

In [20]:
# Function to perform similarity search
def search_similar_documents(query, index, embedding_model, top_k=5):
    """
    Searches for the most relevant documents in the Pinecone index based on query similarity.

    Args:
        query (str): The input legal question.
        index (pinecone.Index): The Pinecone index instance.
        embedding_model (SentenceTransformer): The embedding model.
        top_k (int): Number of top similar documents to retrieve.

    Returns:
        list: Retrieved similar documents.
    """
    query_embedding = embedding_model.encode(query).tolist()
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    return [match['metadata']['text'] for match in results['matches']]

In [21]:
# Function to generate response
import re

def generate_legal_response(question, index, embedding_model, llm, top_k=5):
    """
    Generates a legal response by retrieving relevant documents and using the LLM.

    Args:
        question (str): The legal question asked by the user.
        index (pinecone.Index): The Pinecone index instance.
        embedding_model (SentenceTransformer): The embedding model.
        llm (Pipeline): The LLM text generation pipeline.
        top_k (int): Number of relevant document chunks to retrieve.

    Returns:
        str: The legal response generated by the LLM.
    """
    # Retrieve relevant context from the vector database
    relevant_docs = search_similar_documents(question, index, embedding_model, top_k)

    if not relevant_docs:
        return "I couldn't find relevant legal information in the provided documents."

    context = "\n\n".join(relevant_docs)

    # Prompt template
    prompt = PROMPT_TEMPLATE.format(context=context, question=question)

    # Generate response
    response = llm(prompt, max_new_tokens=256, do_sample=True, temperature=0.7)[0]['generated_text']

    # Remove any parts of the prompt accidentally included in the output
    cleaned_response = response.replace(prompt, "").strip()

    # Remove "Legal Answer:", "Context:", and "Instructions:" titles and content if they appear in the original response
    cleaned_response = re.sub(r"(?i)(Legal Answer:|Context:|Instructions:)", "", cleaned_response).strip()

    return cleaned_response

In [22]:
question = "What were the key legal arguments in the case AFFAIRE C.P. ET M.N. c. FRANCE?"

response = generate_legal_response(question, index, embedding_model, llm, top_k=5)

print(response)

Based on the given context, the key legal arguments in the case AFFAIRE C.P. ET M.N. c. FRANCE were:

1. Legal challenge to paternity: The applicant, C.P., challenged the legally established paternity of the child, N., on the grounds that he was not the biological father. This challenge was based on Article 332 of the Civil Code, which provides for the possibility of challenging paternity based on evidence that the recognized father is not the biological father.
2. Five-year limitation period: The Court held that the action for paternity challenge was inadmissible on the basis of Article 333(2) of the Civil Code, which provides that no one may challenge filiation where possession of a state in conformity with a recognition has lasted for at least five years from birth or the recognition, if it has been made subsequently. The Court found that the child had been brought before the Court five years after the recognition of the legal father, and therefore the action was time-barred.
3. Obl

In [23]:
question = "What was the main issue in the case of C.P. and M.N. v. France?"

response = generate_legal_response(question, index, embedding_model, llm, top_k=5)

print(response)

Answer:
The main issue in the case of C.P. and M.N. v. France was the challenge to paternity. The applicant, C.P., had brought a paternity challenge against the legal father, M.N., on the ground that he was not the biological father of the child. The case revolved around the interpretation of Article 333(2) of the Civil Code, which provides that no one may challenge filiation where possession of a state in conformity with a recognition has lasted for at least five years from birth or the recognition, if it has been made subsequently. The Court of Appeal held that the action for paternity challenge was inadmissible due to the five-year limitation period, and the applicant's former partner had recognised the child before his birth in 2007. Therefore, the main issue was whether the applicant could still challenge the paternity of the child despite the five-year limitation period having passed.


In [24]:
question = "Why did the French courts declare the applicant’s paternity challenge inadmissible?"

response = generate_legal_response(question, index, embedding_model, llm, top_k=5)

print(response)

Answer:
Based on the given context, the French courts declared the applicant's paternity challenge inadmissible due to the expiration of the five-year limitation period for filiation challenges as provided by Article 333(2) of the Civil Code. According to the judgment, the action for paternity challenge was brought before the Court more than five years after the recognition of the child's paternity, which rendered it inadmissible.


In [25]:
question = "What was the European Court of Human Rights’ (ECtHR) decision in this case?"

response = generate_legal_response(question, index, embedding_model, llm, top_k=5)

print(response)

Answer:
Based on the provided context, the ECtHR decision in this case was not explicitly mentioned. However, from the excerpts of the judgments of the Court of Justice of the European Communities and of the Court of Appeal, it can be inferred that the applicant's action for paternity challenge was inadmissible on the basis of Article 333(2) of the Civil Code. The Court held that no one, except the Public Prosecutor's Office, may challenge filiation where possession of a state in conformity with a recognition has lasted for at least five years from birth or the recognition, if it has been made subsequently. Therefore, the ECtHR decision in this case was likely in line with this legal framework and ruled in favor of the state.


In [26]:
!pip install -qU \
  langchain==0.1.1 \
  langchain-community==0.0.13 \
  openai==0.27.7 \
  tiktoken==0.4.0 \
  pinecone-client==3.1.0 \
  pinecone-datasets==0.7.0 \
  pinecone-notebooks==0.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/

In [27]:
import os

if not os.environ.get("PINECONE_API_KEY"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

In [28]:
from pinecone import Pinecone

api_key = getpass("Enter your Pinecone API token: ")

# configure client
pc = Pinecone(api_key=api_key)

Enter your Pinecone API token: ··········


In [29]:
index_name

'legal-llm'

In [30]:
import time

index = pc.Index(index_name)
# wait a moment for connection
time.sleep(1)

index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 17}},
 'total_vector_count': 17}

In [31]:
from langchain_community.embeddings import SentenceTransformerEmbeddings

model_name = "all-MiniLM-L6-v2"  # You can change this to another Sentence Transformer model
embed = SentenceTransformerEmbeddings(model_name=model_name)

In [32]:
from langchain.vectorstores import Pinecone

text_field = "text"

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [33]:
query = "What were the key legal arguments in the case AFFAIRE C.P. ET M.N. c. FRANCE?"

vectorstore.similarity_search(
    query,  # our search query
    k=5  # return 5 most relevant docs
)

[Document(page_content='32495/15, §§ 56-58, 13 October 2020). However, even in the case of limited control, the choices made by the State are not without their examination by the Court. The Court must examine, in the light of the whole case, the grounds on which it has been taken into account ORDER C.P. and M.N. v. FRANCE 11in order to arrive at the solution chosen and to ascertain whether a fair balance has been struck between the different interests involved. In so doing, it must pay particular attention to the essential principle that'),
 Document(page_content='The Court of Appeal, which thus conducted a search allegedly omitted from the balance of interests, legally justified its decision; (...)" THE RELEVANT LEGAL FRAMEWORK AND INTERNAL PRACTICE I. THE NOTION OF STATE HOLDING IN RESPECT OF THE CHILD Article 332 of the Civil Code provides that paternity may be challenged by referring to evidence that the author of the recognition is not the father. JUDGMENT C.P. and M.N. v. FRANCE 

In [34]:
from langchain_community.llms import HuggingFacePipeline

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

# Configure the model for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# Load the model with the BitsAndBytesConfig
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    quantization_config=bnb_config,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [35]:
# Create a pipeline for text generation
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [36]:
# Use HuggingFacePipeline to adapt the HuggingFace pipeline to the Langchain interface
llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [37]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [38]:
query = "What were the key legal arguments in the case ANAGNOSTAKIS v. GREECE?"

result = qa.run(query)

print(result)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Based on the text provided, the key legal arguments in the case of Anagnostakis v. Greece appear to be related to the issue of paternity and the calculation of the five-year period for challenging paternity as provided for in Article 332 of the Civil Code of Greece. The applicants argued that the national courts did not take into account their particular circumstances, including the knowledge by the legal father of the facts relating to the child's paternal filiation, and that the decision-making process did not contain sufficient safeguards to protect their interests. The Court of Appeal's decision was based on a search that was allegedly omitted from the balance of interests, and the Court of Justice of the European Communities and of the Court of Human Rights found that the national courts had struck a fair balance between the various interests involved without affecting the substance of the right to respect for private and family life guaranteed by Article 8 of the Convention.


In [39]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [40]:
qa.run(query)

" Based on the provided context, the key legal arguments in the case Anagnostakis v. Greece were related to the challenge of the father's paternity and the application of the relevant legal framework, specifically Article 332 of the Civil Code and the Court of Appeal's decision. The applicants maintained that the national authorities did not take into account the particular circumstances of the case, including the legal father's knowledge of the facts relating to the child's paternal filiation. The Court of Appeal's decision was based on a detailed examination of the facts and a balance between the different interests at stake, and the Court of Justice of the European Communities and of the Court of Appeal found that the national courts had succeeded in striking a fair balance between the various interests involved. Therefore, there was no violation of Article 8 of the Convention."

In [41]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        )
    )
]

In [42]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [43]:
agent(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Do I need to use a tool? Yes
Action: Knowledge Base
Action Input: ANAGNOSTAKIS v. GREECE
Observation:  The applicants complain of a violation of their right to respect for their private and family life.
Unhelpful Answer: The applicants complain of a violation of their right to respect for their private and family life because they consider that the courts have not sufficiently taken into account the particular circumstances of the case, in particular the knowledge by the legal father of the facts relating to N's paternal filiation.
Thought:

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: ` This answer is unhelpful because it does not provide any legal arguments in support of the applicants' position.

Please provide a more helpful answer.`